In [7]:
import time

from src.swagger import SwaggerParser
from src.config import Config
import json
import os

os.environ["model"] = "gpt-4o-2024-05-13"
os.environ["language_model_key"] = ""

# root_path = "/Users/naariah/Documents/Python_Codes/api-suts/specifications/v3/languageTool.json"
root_path = "/Users/naariah/Documents/Python_Codes/api-suts/specifications/v3/gitlab-groups-13.json"
config = Config()
config.swagger = root_path
parser = SwaggerParser(config)
operations = parser.extract()

2024-07-04 14:25:23.195 | DEBUG    | src.swagger:__init__:28 - Parsing specification file


In [8]:
operations

[get:/groups,
 post:/groups,
 get:/groups/{id},
 put:/groups/{id},
 delete:/groups/{id},
 post:/groups/{id}/share,
 delete:/groups/{id}/share/{group_id},
 get:/groups/{id}/subgroups,
 get:/groups/{id}/hooks,
 post:/groups/{id}/hooks,
 get:/groups/{id}/hooks/{hook_id},
 put:/groups/{id}/hooks/{hook_id},
 delete:/groups/{id}/hooks/{hook_id},
 get:/groups/{id}/descendant_groups,
 get:/groups/{id}/projects,
 post:/groups/{id}/projects/{project_id},
 get:/groups/{id}/projects/shared]

In [9]:
from collections import defaultdict


def create_resource_node(operations):
    def merge_resource(resource, resource_tree):
        for parent, children in resource_tree.items():
            for child in children.copy():
                if child + "s" in children:
                    children.remove(child)
                    if child + "s" in resource:
                        resource[child + "s"].extend(resource[child])
                        del resource[child]

    resource = {}
    resource_tree = defaultdict(set)

    for operation in sorted(operations, key=lambda x: len(x.path.__repr__().split("/"))):
        path = operation.path.__repr__()
        split_path = path.split("/")
        if "{" in split_path[-1]:
            resource_name = split_path[-2]
        else:
            resource_name = split_path[-1]
        if resource_name not in resource:
            resource[resource_name] = []
        resource[resource_name].append(operation.__repr__())

        for previous_resource in resource.keys():
            if previous_resource in split_path and previous_resource != resource_name:
                resource_tree[previous_resource].add(resource_name)

    merge_resource(resource, resource_tree)
    
    return resource, resource_tree

resource, resource_tree = create_resource_node(operations)
resource, resource_tree

({'groups': ['get:/groups',
   'post:/groups',
   'get:/groups/{id}',
   'put:/groups/{id}',
   'delete:/groups/{id}'],
  'share': ['post:/groups/{id}/share', 'delete:/groups/{id}/share/{group_id}'],
  'subgroups': ['get:/groups/{id}/subgroups'],
  'hooks': ['get:/groups/{id}/hooks',
   'post:/groups/{id}/hooks',
   'get:/groups/{id}/hooks/{hook_id}',
   'put:/groups/{id}/hooks/{hook_id}',
   'delete:/groups/{id}/hooks/{hook_id}'],
  'descendant_groups': ['get:/groups/{id}/descendant_groups'],
  'projects': ['get:/groups/{id}/projects',
   'post:/groups/{id}/projects/{project_id}'],
  'shared': ['get:/groups/{id}/projects/shared']},
 defaultdict(set,
             {'groups': {'descendant_groups',
               'hooks',
               'projects',
               'share',
               'shared',
               'subgroups'},
              'projects': {'shared'}}))

In [ ]:
from src.rest import PathParam

info = []
for operation in operations:
    info.append(
                {
                    "name": operation.__repr__(),
                    "description": operation.description,
                }
            )

In [ ]:
info

In [ ]:
operation = operations[3]
info = []
for f in operation.get_leaf_factors():
    info.append(
                {
                    "name": f.get_global_name,
                    "type": f.type.value,
                    "description": f.description,
                }
            )

In [ ]:
from openai import OpenAI
from src.template import *
from src.languagemodel.llm import count_tokens

client = OpenAI(api_key=os.environ["language_model_key"],base_url="https://api.bianxieai.com/v1")
messages = []
messages.append({"role": "system", "content": SystemRole.SYS_ROLE_SEQUENCE})
prompt = INFO.SEQUENCE.format(info) + Task.SEQUENCE
messages.append({"role": "user", "content": prompt})

In [ ]:
messages

In [ ]:
count_tokens(messages, os.environ["model"])

In [ ]:
from loguru import logger

while True:
    try:
        start = time.time()
        response = client.chat.completions.create(
            model=os.environ["model"],
            messages=messages,
            temperature=0.5,
            top_p=0.99,
            frequency_penalty=0,
            presence_penalty=0,
            max_tokens=4096,
            response_format={"type": "json_object"}
        )
        end = time.time()
        print("Time taken: ", end - start)
        break
    except Exception as e:
        logger.error(f"Error in calling language model: {e}")
        logger.error("Retrying...")

In [ ]:
json.loads(response.choices[0].message.content)

In [ ]:
import rstr


def generate_random_string_from_pattern(pattern, min_length=0, max_length=None):
    try:
        generated_str = rstr.xeger(pattern)
        if max_length:
            generated_str = generated_str[min_length:max_length-1]
        return generated_str
    except Exception:
        pass
    return None


pattern = r'[A-Za-z0-9!@#$%^&*()_+]{10,15}'
random_string = generate_random_string_from_pattern(pattern)
print(random_string)